In [3]:
!pip3 install tensorflow
!pip3 install matplotlib
!pip3 install pandas
!pip3 install numpy

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing




ModuleNotFoundError: No module named 'tensorflow'

In [1]:

print('version',tf.__version__)

NameError: name 'tf' is not defined

In [ ]:
data_source = '../dataset/df_AAPL.csv';
file_1 = pd.read_csv(data_source, low_memory=True)
df = pd.DataFrame(file_1)
df

NameError: name 'pd' is not defined